In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from utils.data_helper import *
from utils.data import *
from utils.stats import *
from utils.performance import *
from plotly.subplots import make_subplots
from account.Binance import Binance
from binance.client import Client
from pandas.core.frame import DataFrame
import pandas as pd
import warnings
from strategy_v3.Strategy import GridArithmeticStrategy
from strategy_v3.Executor import ExecutorBinance, ExecutorBacktest
from strategy_v3.DataLoader import DataLoaderBinance
from datetime import datetime, timezone
from time import sleep

pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

In [2]:
symbol = 'BTCUSDT'
binance = Binance()
df = binance.get_historical_instrument_price(symbol, interval='1m', start_str='1 Day ago')
plot_price_ohcl(df, symbol)

# Grid Trading Logic (Arithmetic)

## Example of First Grid Strategy without stoploss and grid resettings

- If I use the same grid without re-scaling, what would be the backtest PnL?

- Limit the size of each grid order to no more than 1-2% of your account balance to reduce the risk of significant losses.

- Backtest method

    - Keep a structure which stores all the limits orders and determine whether it is executed or not
    
    - At each time point, we keep track which orders are executed / not execured / be executed in this periods based on OHCL    

    - Finally, calculate the returns based on limit orders

    - <b>the backtest way is designed in a way that is applicable for all others algo trading strategy based on many limit orders</b>

In [4]:
instrument = 'BTCUSDT'
interval = '5m'
lookback_period = '1 Days ago'
vol_lookback = 30
vol_scale = 1
position_size = 1
binance = Binance()

df = binance.get_historical_instrument_price(instrument, interval=interval, start_str=lookback_period)
df['Vol'] = df['Close'].diff().rolling(vol_lookback).std()
df["half_life"] = df['Close'].rolling(100).apply(lambda x: time_series_half_life(x))
df["hurst_exponent"] = df["Close"].rolling(100).apply(lambda x: time_series_hurst_exponent(x))

start_dt_offset = 150
start_dt_offset = max(start_dt_offset, vol_lookback+1)

start_dt = df['Date'].iloc[start_dt_offset]
start_px = df['Open'].iloc[start_dt_offset]
start_vol = df['Vol'].iloc[start_dt_offset-1]

grid_scales = list(range(-5,5+1,1))
grid_prices = [start_px + x * start_vol * vol_scale for x in grid_scales]

# we keep track an order list for backtest
orders_list = []
for i, px in enumerate(grid_prices):
    if grid_scales[i] == 0:
        continue

    orders_list.append({
        'price': px,
        'qty': position_size,
        'side': 'sell' if grid_scales[i] > 0 else 'buy',
        'order_dt': start_dt,
        'filled_dt': None,
        'filled': False,
        'cancelled': False,
    })

for i, row in df.iterrows():
    date = row['Date']

    if date < start_dt:
        continue

    open, close, high, low = row['Open'], row['Close'], row['High'], row['Low']    
    for i in range(len(orders_list)):
        order = orders_list[i]
        if order['filled'] == True:
            continue
        order_px = order['price']

        # order_px are within the price range in the periods
        if order_px >= low and order_px <= high:
            order['filled'] = True
            order['filled_dt'] = date
        orders_list[i] = order

# after all logic, close out leave the unfilled orders and create an market orders to close out the positions    
df_orders = pd.DataFrame(orders_list)
df_orders['net_qty'] = np.where(df_orders['side'] == 'buy', df_orders['qty'], -1 * df_orders['qty'])
df_orders['cancelled'] = ~df_orders['filled']
net_qty = df_orders[df_orders['filled'] == True]['net_qty'].sum()

if net_qty != 0:
    close_order = pd.DataFrame([{
        'price': close,
        'qty': abs(net_qty),
        'side': 'buy' if net_qty < 0 else 'sell',
        'order_dt': date,
        'filled_dt': date,
        'filled': True,
        'cancelled': False,
        'net_qty': -net_qty

    }])
    df_orders = pd.concat([df_orders, close_order])

df_orders = df_orders.sort_values(['filled_dt'])
display(df_orders)

# calculate the PNL based on filled orders
pnl = df_orders[df_orders['filled'] == True]
pnl['mv'] = -1 * pnl['net_qty'] * pnl['price']
pnl = pnl['mv'].sum()
print(pnl)

fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.1, 
    subplot_titles=[
        'Price OHLC',         
    ],         
    row_heights=[0.8, 0.2, 0.2],  
)
fig.update_layout(
    title=instrument,
    width=1500, height=1000,        
    hovermode='x',
)
colors = plotly.colors.DEFAULT_PLOTLY_COLORS
fig.add_trace(go.Candlestick(x=df["Date"], open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"], name="OHLC", legendgroup='OHCL'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=df['half_life']), row=2,col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=df['hurst_exponent']), row=3,col=1)

for i, grid in enumerate(grid_prices):
    lw = 1
    if grid_scales[i] > 0:
        color = 'green'        
    elif grid_scales[i] < 0:
        color = 'red'        
    else:
        color = 'black'
        lw = 2

    fig.add_hline(y=grid, line_width=lw, line_dash="dash", line_color=color, row=1, col=1)
fig.add_vline(x=start_dt, line_width=1, line_dash="dash", row=1, col=1)
fig.update(layout_xaxis_rangeslider_visible=False)

,price,qty,side,order_dt,filled_dt,filled,cancelled,net_qty
5,52433.408800,1,sell,2024-02-19 07:15:00,2024-02-19 07:15:00,True,False,-1
6,52482.827601,1,sell,2024-02-19 07:15:00,2024-02-19 07:25:00,True,False,-1
4,52334.571200,1,buy,2024-02-19 07:15:00,2024-02-19 08:05:00,True,False,1
3,52285.152399,1,buy,2024-02-19 07:15:00,2024-02-19 08:15:00,True,False,1
2,52235.733599,1,buy,2024-02-19 07:15:00,2024-02-19 08:30:00,True,False,1
0,52136.895998,1,buy,2024-02-19 07:15:00,2024-02-19 08:45:00,True,False,1
1,52186.314798,1,buy,2024-02-19 07:15:00,2024-02-19 08:45:00,True,False,1
0,52094.010000,3,sell,2024-02-19 18:40:00,2024-02-19 18:40:00,True,False,-3
7,52532.246401,1,sell,2024-02-19 07:15:00,NaT,False,True,-1
8,52581.665202,1,sell,2024-02-19 07:15:00,NaT,False,True,-1


19.598407893252443


# A more comprehensive implementation

1. strategy in idle states

2. place grid orders when hurst exponent < 0.5 and half life < k intervals

3. close position when price reach stop loss -> go back to (1)

4. exit strategy when all grids orders are equally cancelled out -> go back to (1)

In [357]:
# Required functions. Real trading should be actual function connect the brokers API
def place_grid_orders(
        df_orders: DataFrame,
        date:datetime, 
        position_size: float,
        curent_px: float, 
        current_vol: float,                
        grid_id: int,
        vol_scale,
        vol_stoploss_scale,
    ) -> DataFrame:
    
    grid_scales = list(range(-5,5+1,1))
    grid_prices = [curent_px + x * current_vol * vol_scale for x in grid_scales]

    stoploss = (curent_px - vol_stoploss_scale * current_vol, curent_px + vol_stoploss_scale * current_vol)

    # we keep track an order list for backtest
    orders_list = []
    for i, px in enumerate(grid_prices):
        if grid_scales[i] == 0:
            continue

        orders_list.append({
            'price': px,
            'qty': position_size,
            'side': 'sell' if grid_scales[i] > 0 else 'buy',
            'order_dt': date,
            'filled_dt': None,
            'filled': False,
            'cancelled': False,
            'net_qty': -position_size if grid_scales[i] > 0 else position_size,
            'grid_id': grid_id,
            'remark': 'grid',
        })

    df_orders = pd.concat([df_orders, pd.DataFrame(orders_list)])
    return df_orders, stoploss

def cancel_all_orders(df_orders: DataFrame) -> DataFrame:
    df_orders['cancelled'] = df_orders['filled'] == False
    return df_orders

def close_out_position(
        df_orders: DataFrame, 
        date: datetime,
        close: float,
        grid_id: int,
        remark: str,
        ) -> DataFrame:    
    
    net_qty = df_orders[df_orders['filled'] == True]['net_qty'].sum()    
    if net_qty != 0:
        close_order = pd.DataFrame([{
            'price': close,
            'qty': abs(net_qty),
            'side': 'buy' if net_qty < 0 else 'sell',
            'order_dt': date,
            'filled_dt': date,
            'filled': True,
            'cancelled': False,
            'net_qty': -net_qty,
            'grid_id': grid_id,
            'remark': remark,
        }])
        df_orders = pd.concat([df_orders, close_order])
        
    return df_orders

def is_position_natural(df_orders: DataFrame) -> bool:
    last_grid = df_orders[df_orders['grid_id'] == df_orders['grid_id'].max()]
        
    filled = last_grid[last_grid['filled'] == True]
    has_filled = len(filled) > 0
    filled_net_qty = filled['net_qty'].sum()

    if has_filled and filled_net_qty == 0:
        return True
    
    return False

# this only for backtesting to replicate the fills, in reality, this is filled by brokers
def fill_orders(
        df_orders: DataFrame, 
        date:datetime, 
        low:float,
        high:float,         
        ) -> DataFrame:
    
    fill_filters = (df_orders['filled'] == False)&\
        (df_orders['cancelled'] == False)&\
        (df_orders['price'] >= low) &\
        (df_orders['price'] <= high)   

    df_orders['filled'] = np.where(fill_filters, True, df_orders['filled'])
    df_orders['filled_dt'] = pd.to_datetime(np.where(fill_filters, date, df_orders['filled_dt']))                
    return df_orders

def is_idle(df_orders: DataFrame):
    x = df_orders.copy()
    x = x[x['filled'] == False]
    x = x[x['cancelled'] == False]    
    is_idle = len(x) == 0
    return is_idle

def compute_pnl(df_orders):
    pnl = df_orders[df_orders['filled'] == True]
    assert pnl['net_qty'].sum() == 0

    pnl['mv'] = -1 * pnl['net_qty'] * pnl['price']    
    pnl_gross = pnl['mv'].sum()    
    fee = pnl['transaction_fee'].sum()    
    pnl_net = pnl_gross - fee

    return pnl_net

In [388]:
instrument = 'BTCUSDT'
interval = '5m'
lookback_period = '10 Days ago'

lookback = 30
vol_scale = 0.6
vol_stoploss_scale = 7
position_size = 1
binance = Binance()

df = binance.get_historical_instrument_price(instrument, interval=interval, start_str=lookback_period)
df['Vol'] = df['Close'].diff().rolling(lookback).std().shift(1)
df["half_life"] = df['Close'].rolling(100).apply(lambda x: time_series_half_life(x)).shift(1)
df["hurst_exponent"] = df["Close"].rolling(100).apply(lambda x: time_series_hurst_exponent(x)).shift(1)

In [389]:
grid_id = 0
df_orders = pd.DataFrame(columns=['price', 'qty', 'side', 'order_dt', 'filled_dt', 'filled', 'cancelled', 'net_qty', 'grid_id', 'remark'])
stoploss = (-float('inf'), float('inf'))

for i, row in df.iterrows():
    date = row['Date']
    hurst_exponent = row['hurst_exponent']
    open, close, high, low = row['Open'], row['Close'], row['High'], row['Low']            
    vol = row['Vol']    

    # at beginning of periods, use open price to create grid orders
    if is_idle(df_orders) and hurst_exponent < 0.5:
        grid_id += 1
        df_orders, stoploss = place_grid_orders(df_orders, date, position_size, open, vol, grid_id, vol_scale, vol_stoploss_scale)        

    # fill the orders using high and low        
    df_orders = fill_orders(df_orders, date, low, high)        

    # after filling, if position is netural, cancel all orders
    if is_position_natural(df_orders):                
        df_orders = cancel_all_orders(df_orders)        

    # finally, check the close price and determine if we need stop loss
    if not is_idle(df_orders) and (close < stoploss[0] or close > stoploss[1]):        
        df_orders = cancel_all_orders(df_orders)
        df_orders = close_out_position(df_orders, date, close, grid_id, 'stoploss')
    
df_orders = cancel_all_orders(df_orders)
df_orders = close_out_position(df_orders, date, close, grid_id, 'close')
df_orders['transaction_fee'] = np.where(df_orders['filled'] == True, df_orders['qty'] * df_orders['price'] * 0/100, 0)

pnl = compute_pnl(df_orders)
return_pct = pnl/(close*10)

print('PnL: ${:,.0f}, Return: {:.2f}%, num of orders: {:.2f}'.format(pnl, return_pct*100, len(df_orders)))
df_orders = df_orders.sort_values(['grid_id', 'filled_dt'])

PnL: $4,356, Return: 0.86%, num of orders: 659.00


In [ ]:
fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.1, 
    subplot_titles=[
        'Price OHLC',     
        'Half Life',
        'Hurst Exponent',
    ],         
    row_heights=[0.8, 0.2, 0.2],  
)
fig.update_layout(
    title=instrument,
    width=1500, height=1000,        
    hovermode='x',
)
colors = plotly.colors.DEFAULT_PLOTLY_COLORS
fig.add_trace(go.Candlestick(x=df["Date"], open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"], name="OHLC", legendgroup='OHCL'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=df['half_life']), row=2,col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=df['hurst_exponent']), row=3,col=1)

for i in range(df_orders['grid_id'].max()):
    temp = df_orders[df_orders['grid_id'] == i+1]
    grid_min_dt = temp['order_dt'].min()
    grid_max_dt = temp['filled_dt'].max()
    grid_prices = sorted(temp[temp['remark'] == 'grid']['price'].values)

    for i, grid in enumerate(grid_prices):
        lw = 1
        if i/2 >= 2.5:
            color = 'green'        
        else:
            color = 'red'  

        fig.add_shape(type='line', x0=grid_min_dt, x1=grid_max_dt, y0=grid, y1=grid, line=dict(color=color, dash='dash'), row=1, col=1)

filled_orders = df_orders[df_orders['filled'] == True]
filled_orders_buy = df_orders[df_orders['side'] == 'buy']
filled_orders_sell = df_orders[df_orders['side'] == 'sell']

fig.add_trace(go.Scatter(x=filled_orders_buy['filled_dt'], y=filled_orders_buy['price'], marker=dict(color='green',size=15), mode='markers', marker_symbol=5, name='Buy'),row=1, col=1)
fig.add_trace(go.Scatter(x=filled_orders_sell['filled_dt'], y=filled_orders_sell['price'], marker=dict(color='red',size=15), mode='markers', marker_symbol=6, name='Sell'),row=1, col=1)
fig.update(layout_xaxis_rangeslider_visible=False)

# Strategy Class

## BackTest Strategy

In [2]:
strategy = GridArithmeticStrategy(
    instrument = 'BTCUSDT',
    interval = '5m',
    grid_size = 5,
    vol_lookback = 30,
    vol_grid_scale = 0.4,
    vol_stoploss_scale = 7,
    position_size = 100,
    hurst_exp_threshold = 0.5
)
strategy.set_price_decimal(2)
strategy.set_qty_decimal(5)
strategy.set_data_loder(DataLoaderBinance())
strategy.set_executor(ExecutorBacktest())
strategy.load_data('1 Days Ago')

In [3]:
df = strategy.df.copy()
for i, data in df.iterrows():
    strategy.execute(data)
strategy.close_out_positions('close', data['Close'], data['Date'])
strategy.summary(True)

2024-02-24 03:12:30,192 - grid_995789 - INFO - creating 10 grid orders of 0.00196 BTCUSDT at grid center price 51046.92....
2024-02-24 03:12:30,202 - grid_995789 - INFO - [2024-02-23 03:35:00] filling 1 orders...
2024-02-24 03:12:30,208 - grid_995789 - INFO - state: ACTIVE, hurst_exponent is 0.53 (above threshold 0.50).
2024-02-24 03:12:30,209 - grid_995789 - INFO - [2024-02-23 03:40:00] filling 3 orders...
2024-02-24 03:12:30,214 - grid_995789 - INFO - state: ACTIVE, hurst_exponent is 0.55 (above threshold 0.50).
2024-02-24 03:12:30,220 - grid_995789 - INFO - state: ACTIVE, hurst_exponent is 0.55 (above threshold 0.50).
2024-02-24 03:12:30,226 - grid_995789 - INFO - state: ACTIVE, hurst_exponent is 0.55 (above threshold 0.50).
2024-02-24 03:12:30,231 - grid_995789 - INFO - state: ACTIVE, hurst_exponent is 0.55 (above threshold 0.50).
2024-02-24 03:12:30,235 - grid_995789 - INFO - state: ACTIVE, hurst_exponent is 0.56 (above threshold 0.50).
2024-02-24 03:12:30,240 - grid_995789 - INFO

KeyboardInterrupt: 

# Execute Strategy

In [738]:
strategy = GridArithmeticStrategy(
    instrument = 'BTCUSDT',
    interval = '5m',
    grid_size = 5,
    vol_lookback = 30,
    vol_grid_scale = 0.4,
    vol_stoploss_scale = 7,
    position_size = 20,
    hurst_exp_threshold = 0.5
)

strategy = GridArithmeticStrategy(
    instrument = 'BTCUSDT',
    interval = '5m',
    grid_size = 5,
    vol_lookback = 30,
    vol_grid_scale = 5,
    vol_stoploss_scale = 7,
    position_size = 10,
    hurst_exp_threshold = 1
)

strategy.set_price_decimal(2)
strategy.set_qty_decimal(5)
strategy.set_data_loder(DataLoaderBinance())
strategy.set_executor(ExecutorBinance())

In [740]:
while True:
    try:
        strategy.load_data('1 Days Ago')
        df = strategy.df.copy()
        data = df.iloc[-1]
        dt_now = pd.to_datetime(datetime.now(timezone.utc)).replace(tzinfo=None)
        interval = df['Date'].diff().iloc[-1].seconds
        since_last = (dt_now - data['Date']).seconds
        assert since_last <= interval
        strategy.execute(data)        
        sleep(30)
    except:
        strategy.cancel_all_orders()
        strategy.close_out_positions('close', data['Close'], data['Date'])

2024-02-23 02:27:24,188 - grid_837465 - INFO - creating 10 grid orders of 0.00019 BTCUSDT at grid center price 51627.99....
2024-02-23 02:27:27,620 - grid_837465 - INFO - cancelling all orders.....
2024-02-23 02:27:28,675 - grid_837465 - INFO - nothing to close out because of no oustanding positions....
2024-02-23 02:27:29,065 - grid_837465 - INFO - creating 10 grid orders of 0.00019 BTCUSDT at grid center price 51627.99....
2024-02-23 02:27:40,459 - grid_837465 - INFO - cancelling all orders.....
2024-02-23 02:27:41,546 - grid_837465 - INFO - nothing to close out because of no oustanding positions....
2024-02-23 02:27:41,948 - grid_837465 - INFO - creating 10 grid orders of 0.00019 BTCUSDT at grid center price 51621.37....


In [735]:
strategy.close_out_positions()

2024-02-23 02:11:53,885 - grid_982381 - INFO - [2024-02-22 18:11:53] nothing to close out because no oustanding positions....


In [5]:
input('s')

'dd'